# 全局参数设置

In [1]:
# 全局变量
import os
os.environ['HF_HOME'] = 'E:\PycharmProjects\huggingface'
os.environ['HF_HUB_CACHE'] = 'E:\PycharmProjects\huggingface\cache'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

model_name_or_path = "models/whisper-large-v2"
model_dir = "models/whisper-large-v2-asr-int8"

language = "Chinese (China)"
language_abbr = "zh-CN"
task = "transcribe"
dataset_name = "mozilla-foundation/common_voice_11_0"

batch_size=20

In [2]:
# 全局 import

from datasets import load_dataset, DatasetDict, Audio
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor,AutoModelForSpeechSeq2Seq,Seq2SeqTrainingArguments,Seq2SeqTrainer
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model,prepare_model_for_int8_training
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 准备数据集

In [3]:
# 准备数据集 Common Voice

common_voice = DatasetDict()
common_voice["train"] = load_dataset(dataset_name, language_abbr, split="train",trust_remote_code=True)
common_voice["validation"] = load_dataset(dataset_name, language_abbr, split="validation",trust_remote_code=True)

# 移除数据集中不必要的字段
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
)

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

# 准备模型

In [4]:
# 从预训练模型加载特征提取器
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)

# 从预训练模型加载分词器，可以指定语言和任务以获得最适合特定需求的分词器配置
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

# 从预训练模型加载处理器，处理器通常结合了特征提取器和分词器，为特定任务提供一站式的数据预处理
processor = AutoProcessor.from_pretrained(model_name_or_path, language=language, task=task)

# 使用 int8 精度加载预训练模型，进一步降低显存需求。
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")

# 设置模型配置中的 forced_decoder_ids 属性为 None
model.config.forced_decoder_ids = None  # 这通常用于指定在解码（生成文本）过程中必须使用的特定token的ID，设置为None表示没有这样的强制要求
# 设置模型配置中的 suppress_tokens 列表为空
model.config.suppress_tokens = []  # 这用于指定在生成过程中应被抑制（不生成）的token的列表，设置为空列表表示没有要抑制的token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# 数据处理方法定义：
#  1. 通过加载音频列将音频输入重新采样为16kHZ。
#  2. 使用特征提取器从音频数组计算输入特征。
#  3. 将句子列标记化为输入标签。

def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

tokenized_common_voice = common_voice.map(prepare_dataset)

In [6]:
# PEFT 微调前的模型处理
# 将所有非 int8 精度模块转换为全精度（fp32）以保证稳定性
# 为输入嵌入层添加一个 forward_hook，以启用输入隐藏状态的梯度计算
# 启用梯度检查点以实现更高效的内存训练
# 使用 peft 库预定义的工具函数 prepare_model_for_int8_training，便可自动完成以上模型处理工作。
model = prepare_model_for_int8_training(model)

# LoRA Adapter 配置
# 在 peft 中使用LoRA非常简捷，借助 PeftModel抽象，我们可以快速使用低秩适配器（LoRA）到任意模型。

# 通过使用 peft 中的 get_peft_model 工具函数来实现。

# 关于 LoRA 超参数的说明：
# MatMul(B,A) * Scaling
# Scaling = LoRA_Alpha / Rank

# 创建一个LoraConfig对象，用于设置LoRA（Low-Rank Adaptation）的配置参数
config = LoraConfig(
    r=4,  # LoRA的秩，影响LoRA矩阵的大小
    lora_alpha=64,  # LoRA适应的比例因子
    # 指定将LoRA应用到的模型模块，通常是attention和全连接层的投影。
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,  # 在LoRA模块中使用的dropout率
    bias="none",  # 设置bias的使用方式，这里没有使用bias
)

peft_model = get_peft_model(model, config)
peft_model.config.use_cache = False
# 打印 LoRA 微调训练的模型参数
peft_model.print_trainable_parameters()

trainable params: 1,966,080 || all params: 1,545,271,040 || trainable%: 0.12723204856023188


C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\peft\utils\other.py:141: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


# 准备训练参数

In [7]:
# 定义了一个针对语音到文本（Seq2Seq）模型的自定义数据整理器类，特别适用于输入为语音特征、输出为文本序列的数据集。
# 这个整理器（DataCollatorSpeechSeq2SeqWithPadding）旨在将数据点批量打包，将每个批次中的attention_mask填充到最大长度，
# 以保持批处理中张量形状的一致性，并用-100替换填充值，以便在损失函数中被忽略。这对于神经网络的高效训练至关重要。

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any  # 处理器结合了特征提取器和分词器

    # 整理器函数，将特征列表处理成一个批次
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 从特征列表中提取输入特征，并填充以使它们具有相同的形状
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # 从特征列表中提取标签特征（文本令牌），并进行填充
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 使用-100替换标签中的填充区域，-100通常用于在损失计算中忽略填充令牌
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 如果批次中的所有序列都以句子开始令牌开头，则移除它
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        # 将处理过的标签添加到批次中
        batch["labels"] = labels

        return batch  # 返回最终的批次，准备好进行训练或评估

# 实例化数据整理器
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [8]:
# 设置序列到序列模型训练的参数
training_args = Seq2SeqTrainingArguments(
    output_dir=model_dir,  # 指定模型输出和保存的目录
    per_device_train_batch_size=batch_size,  # 每个设备上的训练批量大小
    learning_rate=1e-3,  # 学习率
    num_train_epochs=1,  # 训练的总轮数
    evaluation_strategy="epoch",  
    # eval_steps=140,
    # logging_steps=140,  # 指定日志记录的步骤，用于跟踪训练进度
    # warmup_steps=50,  # 在训练初期增加学习率的步数，有助于稳定训练
    fp16=True,  # 启用混合精度训练，可以提高训练速度，同时减少内存使用
    per_device_eval_batch_size=batch_size,  # 每个设备上的评估批量大小
    generation_max_length=128,  # 生成任务的最大长度
    remove_unused_columns=False,  # 是否删除不使用的列，以减少数据处理开销
    label_names=["labels"],  # 指定标签列的名称，用于训练过程中
)

# 实例化 Seq2SeqTrainer 训练器
trainer = Seq2SeqTrainer(
    args=training_args,
    model=peft_model,
    train_dataset=tokenized_common_voice["train"],
    eval_dataset=tokenized_common_voice["validation"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
)

# 启动训练

In [9]:
# 开始训练
trainer.train()
trainer.save_model(model_dir)
peft_model.eval()

C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\torch\utils\checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\bitsandbytes\autograd\_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\

Epoch,Training Loss,Validation Loss
1,0.371900,0.380660


C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\torch\utils\checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\bitsandbytes\autograd\_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\

PeftModel(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 1280)
          (layers): ModuleList(
            (0-31): 32 x WhisperEncoderLayer(
              (self_attn): WhisperSdpaAttention(
                (k_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=False)
                (v_proj): lora.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1280, out_features=4, bias=False)
                  )
            